In [ ]:

!pip install git+https://github.com/huggingface/transformers
!pip install accelerate

# for GPU use
!CT_CUBLAS=1 pip install transformers --no-binary transformers


In [2]:
import torch

from transformers import AutoTokenizer, AutoModelForCausalLM
from torch.utils.checkpoint import checkpoint


In [ ]:

model_path = "inception-mbzuai/jais-13b"

device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained(model_path, offload_folder="offload")
model = AutoModelForCausalLM.from_pretrained(model_path, device_map="auto", trust_remote_code=True, offload_folder="offload")



Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

In [ ]:

def get_response(text,tokenizer=tokenizer,model=model):
    input_ids = tokenizer(text, return_tensors="pt").input_ids
    inputs = input_ids.to(device)
    input_len = inputs.shape[-1]
    generate_ids = model.generate(
        inputs,
        top_p=0.9,
        temperature=0.3,
        max_length=200-input_len,
        min_length=input_len + 4,
        repetition_penalty=1.2,
        do_sample=True,
    )
    response = tokenizer.batch_decode(
        generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True
    )[0]
    return response


In [ ]:

text= "عاصمة دولة الإمارات العربية المتحدة ه"
print(get_response(text))


In [ ]:

text = "The capital of UAE is"
print(get_response(text))